# ZAF043_BA_Crowd-Anomaly-Detection 

## Overview
Crowd anomaly detection is a type of machine learning model that is designed to detect unusual behavior in crowds of people. This can be useful for a variety of applications, including security, public safety, and crowd management.

The basic idea behind a crowd anomaly detection model is to use machine learning algorithms to analyze video footage or other types of sensor data in order to identify patterns of behavior that are outside the norm. These patterns could include things like sudden movements, large crowds gathering in unusual places, or groups of people moving in ways that are inconsistent with normal crowd behavior.

## Methodology
- Collect Data: The first step in building a crowd anomaly detection model is to collect a large dataset of crowd behavior. This data can be collected using sensors such as cameras, microphones, or other IoT devices.

- Preprocess Data: Once the data is collected, it needs to be preprocessed to remove any noise or outliers that may interfere with the analysis. This can involve tasks such as filtering, smoothing, and normalization.

- Data Annotation: In order to train a deep learning model, the dataset needs to be labeled with appropriate annotations that indicate normal and abnormal behavior. This can be done manually by human annotators or using automated tools.

- Split Data: The labeled dataset is then split into training, validation, and testing sets. The training set is used to train the model, the validation set is used to optimize the hyperparameters of the model, and the testing set is used to evaluate the performance of the model.

- Build Model: Once the data is preprocessed and split, the next step is to build a deep learning model. Convolutional Neural Networks (CNN) are commonly used for crowd anomaly detection tasks due to their ability to learn spatial features from images or videos.

- Train Model: The model is then trained using the labeled training dataset. This involves repeatedly presenting the model with input data and adjusting the model's parameters to minimize the error between the predicted outputs and the ground truth.

- Validate Model: After the model is trained, it is validated on the validation dataset to ensure that it is not overfitting to the training data. This involves monitoring the model's performance on the validation set and adjusting the model's hyperparameters as needed.

- Test Model: Finally, the model is tested on the testing dataset to evaluate its performance on new, unseen data. The performance metrics of the model, such as accuracy, precision, recall, F1-score, etc., are computed and analyzed.

- Deploy Model: If the model performs well on the testing dataset, it can be deployed in the real world to detect crowd anomalies in real-time. This involves integrating the model with sensors and other IoT devices and developing a user interface for the end-users to visualize and analyze the model's outputs.

## Dataset
- [Avenue Dataset](http://www.cse.cuhk.edu.hk/leojia/projects/detectabnormal/dataset.html)

## Papers
[Abnormal Event Detection in Videos using Spatiotemporal Autoencoder](https://arxiv.org/abs/1701.01546)

## 1. Load data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
from tensorflow.keras.models import load_model

In [4]:
from tensorflow.keras.preprocessing.image import img_to_array,load_img
import numpy as np
import glob
import os 
from skimage import data, color
from skimage.transform import rescale, resize, downscale_local_mean
import argparse
from PIL import Image


train_imagestore=[]



train_video_source_path='/content/drive/MyDrive/Zummit/training_videos'
fps=5
#fps refers to the number of seconds after which one frame will be taken . fps=5 means 1 frame after every 5 seconds. More like seconds per frame.

def create_dir(path):
	if not os.path.exists(path):
		os.makedirs(path)

def remove_old_images(path):
	filelist = glob.glob(os.path.join(path, "*.png"))
	for f in filelist:
		os.remove(f)

def store(image_path):
	img=load_img(image_path)
	img=img_to_array(img)


	#Resize the Image to (227,227,3) for the network to be able to process it. 
	img=resize(img,(227,227,3))

	#Convert the Image to Grayscale
	gray=0.2989*img[:,:,0]+0.5870*img[:,:,1]+0.1140*img[:,:,2]

	train_imagestore.append(gray)



#List of all Videos in the Source Directory.
videos=os.listdir(train_video_source_path)
print("Found ",len(videos)," training videos")


#Make a temp dir to store all the frames
create_dir(train_video_source_path+'/frames')

#Remove old images
remove_old_images(train_video_source_path+'/frames')

framepath=train_video_source_path+'/frames'

for video in videos:
		os.system( 'ffmpeg -i {}/{} -r 1/{}  {}/frames/%03d.jpg'.format(train_video_source_path,video,fps,train_video_source_path))
		images=os.listdir(framepath)
		for image in images:
			image_path=framepath+ '/'+ image
			store(image_path)


train_imagestore=np.array(train_imagestore)
a,b,c=train_imagestore.shape
#Reshape to (227,227,batch_size)
train_imagestore.resize(b,c,a)
#Normalize
train_imagestore=(train_imagestore-train_imagestore.mean())/(train_imagestore.std())
#Clip negative Values
train_imagestore=np.clip(train_imagestore,0,1)
np.save('trainer.npy',train_imagestore)
#Remove Buffer Directory
os.system('rm -r {}'.format(framepath))
print("Program ended. Please wait while trainer.npy is created. \nRefresh when needed")
print('Number of frames created :', int(len(train_imagestore)))

Found  16  training videos
Program ended. Please wait while trainer.npy is created. 
Refresh when needed
Number of frames created : 227


In [5]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import numpy as np 
import argparse
from tensorflow.keras.layers import Conv3D,ConvLSTM2D,Conv3DTranspose
from tensorflow.keras.models import Sequential

''' The following load_model function code has been taken from 
Abnormal Event Detection in Videos using Spatiotemporal Autoencoder
by Yong Shean Chong Yong Haur Tay
Lee Kong Chian Faculty of Engineering Science, Universiti Tunku Abdul Rahman, 43000 Kajang, Malaysia.
It's main purpose is to help us generate the anomaly detector model
'''

#load_model starts here :----------------------------------------------------
def load_model():
	"""
	Return the model used for abnormal event 
	detection in videos using spatiotemporal autoencoder

	"""
	model=Sequential()
	model.add(Conv3D(filters=128,kernel_size=(11,11,1),strides=(4,4,1),padding='valid',input_shape=(227,227,10,1),activation='tanh'))
	model.add(Conv3D(filters=64,kernel_size=(5,5,1),strides=(2,2,1),padding='valid',activation='tanh'))



	model.add(ConvLSTM2D(filters=64,kernel_size=(3,3),strides=1,padding='same',dropout=0.4,recurrent_dropout=0.3,return_sequences=True))

	
	model.add(ConvLSTM2D(filters=32,kernel_size=(3,3),strides=1,padding='same',dropout=0.3,return_sequences=True))


	model.add(ConvLSTM2D(filters=64,kernel_size=(3,3),strides=1,return_sequences=True, padding='same',dropout=0.5))




	model.add(Conv3DTranspose(filters=128,kernel_size=(5,5,1),strides=(2,2,1),padding='valid',activation='tanh'))
	model.add(Conv3DTranspose(filters=1,kernel_size=(11,11,1),strides=(4,4,1),padding='valid',activation='tanh'))

	model.compile(optimizer='adam',loss='mean_squared_error',metrics=['accuracy'])

	return model

#load_model ends here :----------------------------------------------------



X_train=np.load('trainer.npy')
frames=X_train.shape[2]
#Need to make number of frames divisible by 10 to ease the load_model


frames=frames-frames%10

X_train=X_train[:,:,:frames]
X_train=X_train.reshape(-1,227,227,10)
X_train=np.expand_dims(X_train,axis=4)
Y_train=X_train.copy()


epochs=100
batch_size=1

In [7]:
model=load_model()

callback_save = ModelCheckpoint("AnomalyDetector.h5",
                monitor="mean_squared_error")

callback_early_stopping = EarlyStopping(monitor='loss', patience=3)

print('Trainer has been loaded')
model.fit(X_train,Y_train,
      batch_size=batch_size,
      epochs=epochs,
      callbacks = [callback_save,callback_early_stopping]
      )

Trainer has been loaded
Epoch 1/100
20/20 [==============================] - 55s 2s/step - loss: 0.2312 - accuracy: 0.5254
Epoch 2/100
20/20 [==============================] - 45s 2s/step - loss: 0.2064 - accuracy: 0.5457
Epoch 3/100
20/20 [==============================] - 46s 2s/step - loss: 0.2020 - accuracy: 0.5458
Epoch 4/100
20/20 [==============================] - 46s 2s/step - loss: 0.2004 - accuracy: 0.5458
Epoch 5/100
20/20 [==============================] - 46s 2s/step - loss: 0.1733 - accuracy: 0.5827
Epoch 6/100
20/20 [==============================] - 45s 2s/step - loss: 0.1244 - accuracy: 0.6707
Epoch 7/100
20/20 [==============================] - 45s 2s/step - loss: 0.1054 - accuracy: 0.6943
Epoch 8/100
20/20 [==============================] - 45s 2s/step - loss: 0.0961 - accuracy: 0.7044
Epoch 9/100
20/20 [==============================] - 45s 2s/step - loss: 0.0911 - accuracy: 0.7121
Epoch 10/100
20/20 [==============================] - 46s 2s/step - loss: 0.0881 - ac

In [9]:
test_imagestore=[]



test_video_source_path='/content/drive/MyDrive/Zummit/testing_videos'
fps=5
#fps refers to the number of seconds after which one frame will be taken . fps=5 means 1 frame after every 5 seconds. More like seconds per frame.

def create_dir(path):
	if not os.path.exists(path):
		os.makedirs(path)

def remove_old_images(path):
	filelist = glob.glob(os.path.join(path, "*.png"))
	for f in filelist:
		os.remove(f)

def store(image_path):
	img=load_img(image_path)
	img=img_to_array(img)


	#Resize the Image to (227,227,3) for the network to be able to process it. 


	img=resize(img,(227,227,3))

	#Convert the Image to Grayscale


	gray=0.2989*img[:,:,0]+0.5870*img[:,:,1]+0.1140*img[:,:,2]

	test_imagestore.append(gray)
#List of all Videos in the Source Directory.
videos=os.listdir(test_video_source_path)
print("Found ",len(videos)," testing videos")


#Make a temp dir to store all the frames
create_dir(test_video_source_path+'/frames')

#Remove old images
remove_old_images(test_video_source_path+'/frames')

framepath=test_video_source_path+'/frames'
total=0
video_count=0

for video in videos:
		video_count+=1
		print("Video number: ",video_count)
		print("Video:",str(video))
		image_count=0
		os.system( 'ffmpeg -i {}/{} -r 1/{}  {}/frames/%03d.jpg'.format(test_video_source_path,video,fps,test_video_source_path))
		images=os.listdir(framepath)
		image_count=len(images)
		for image in images:
			image_path=framepath+ '/'+ image
			store(image_path)
		total=len(images)+total
		print("Number of images:",image_count,"\n----------\n")


test_imagestore=np.array(test_imagestore)
a,b,c=test_imagestore.shape
#Reshape to (227,227,batch_size)
test_imagestore.resize(b,c,a)
#Normalize
test_imagestore=(test_imagestore-test_imagestore.mean())/(test_imagestore.std())
#Clip negative Values
test_imagestore=np.clip(test_imagestore,0,1)
np.save('tester.npy',test_imagestore)
#Remove Buffer Directory
os.system('rm -r {}'.format(framepath))

print("Program ended. All testing videos shall be stored in tester.npy \n Please wait while tester.npy is created. \nRefresh when needed")
print('Number of frames created :', int(total))
print ('Number of bunches=',int(total),"/10 = ",int(total/10))
print("\nCorrupted and unreadable bunches were ignored")

Found  22  testing videos
Video number:  1
Video: 13.avi
Number of images: 6 
----------

Video number:  2
Video: 21.avi
Number of images: 6 
----------

Video number:  3
Video: 16.avi
Number of images: 8 
----------

Video number:  4
Video: 20.avi
Number of images: 8 
----------

Video number:  5
Video: 19.avi
Number of images: 8 
----------

Video number:  6
Video: 06.avi
Number of images: 12 
----------

Video number:  7
Video: 10.avi
Number of images: 12 
----------

Video number:  8
Video: 04.avi
Number of images: 12 
----------

Video number:  9
Video: 08.avi
Number of images: 12 
----------

Video number:  10
Video: 18.avi
Number of images: 12 
----------

Video number:  11
Video: 05.avi
Number of images: 12 
----------

Video number:  12
Video: 17.avi
Number of images: 12 
----------

Video number:  13
Video: 02.avi
Number of images: 12 
----------

Video number:  14
Video: 09.avi
Number of images: 12 
----------

Video number:  15
Video: 03.avi
Number of images: 12 
----------

In [11]:
def mean_squared_loss(x1,x2):


	''' Compute Euclidean Distance Loss  between 
	input frame and the reconstructed frame'''


	diff=x1-x2
	a,b,c,d,e=diff.shape
	n_samples=a*b*c*d*e
	sq_diff=diff**2
	Sum=sq_diff.sum()
	dist=np.sqrt(Sum)
	mean_dist=dist/n_samples

	return mean_dist

In [12]:
threshold=0.00040

model=load_model('AnomalyDetector.h5')

X_test=np.load('tester.npy')
frames=X_test.shape[2]
#Need to make number of frames divisible by 10


flag=0 #Overall video flagq

frames=frames-frames%10

X_test=X_test[:,:,:frames]
X_test=X_test.reshape(-1,227,227,10)
X_test=np.expand_dims(X_test,axis=4)
counter =0
for number,bunch in enumerate(X_test):
	n_bunch=np.expand_dims(bunch,axis=0)
	reconstructed_bunch=model.predict(n_bunch)


	loss=mean_squared_loss(n_bunch,reconstructed_bunch)
	
	if loss>threshold:
		print("Anomalous bunch of frames at bunch number {}".format(number))
		counter=counter+1
		print("bunch number: ",counter)
		flag=1


	else:
		print('No anomaly')
		counter=counter+1
		print("bunch number: ",counter)



if flag==1:
	print("Anomalous Events detected")
else:
	print("No anomaly detected")
	
print("\nCorrupted and unreadable bunches were ignored")

1/1 [==============================] - 53s 53s/step
No anomaly
bunch number:  1
1/1 [==============================] - 1s 546ms/step
No anomaly
bunch number:  2
1/1 [==============================] - 1s 573ms/step
Anomalous bunch of frames at bunch number 2
bunch number:  3
1/1 [==============================] - 1s 548ms/step
Anomalous bunch of frames at bunch number 3
bunch number:  4
1/1 [==============================] - 1s 539ms/step
No anomaly
bunch number:  5
1/1 [==============================] - 1s 569ms/step
No anomaly
bunch number:  6
1/1 [==============================] - 1s 567ms/step
No anomaly
bunch number:  7
1/1 [==============================] - 1s 546ms/step
No anomaly
bunch number:  8
1/1 [==============================] - 1s 586ms/step
No anomaly
bunch number:  9
1/1 [==============================] - 1s 693ms/step
No anomaly
bunch number:  10
1/1 [==============================] - 1s 1s/step
No anomaly
bunch number:  11
1/1 [==============================] - 1s 1s/